In [1]:
import configparser
config = configparser.ConfigParser() #init
config.read('../configs.ini') # init config with values from configs.ini
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# VARIABLES
# experiment of interest (experiment name must match the section name in configs.ini)
#experiment = 'loneliness'
#experiment = 'incontinence_v5'
#experiment = 'mobility_v5.C.original'
#experiment = 'Falling_NER_v3_20231114'
#experiment = 'Mobility_2404_20240619'
#experiment = 'Loneliness_beta0_20231123'
experiment = 'Incontinence_NER_v5_20231208'

# CONSTANTS from config
# to save data
data_folder = config[experiment]['data_folder'] # must exist
data_subfolder = config[experiment]['data_subfolder'] # must exist
# path to save folder
data_save_folder_path = os.path.join(data_folder, data_subfolder)

In [3]:
dataset = pd.read_parquet(os.path.join(data_save_folder_path, 'tmp_BIO_labeled_data.parquet'))
print(len(dataset))
dataset = dataset[dataset['words'].apply(len) <= 512]
print(len(dataset))
dataset.reset_index(drop=True, inplace=True)

2561
2561


In [4]:
# Get the unique original_ids
unique_ids = dataset['text_id'].unique()

# Split the unique_ids into training, test, and validation using a fixed seed for reproducibility
train_ids, temp_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
test_ids, val_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

# Using these IDs, filter the original dataset to get the respective data splits
train_data = dataset[dataset['text_id'].isin(train_ids)]
test_data = dataset[dataset['text_id'].isin(test_ids)]
val_data = dataset[dataset['text_id'].isin(val_ids)]

In [5]:
# some statistics
print(len(train_data))
print(len(val_data))
print(len(test_data))

print("\nAs percentages")
print((train_data.shape[0]/unique_ids.shape[0]) *100)
print((test_data.shape[0]/unique_ids.shape[0]) * 100)
print((val_data.shape[0]/unique_ids.shape[0])*100)

2048
257
256

As percentages
79.96876220226474
9.996095275283093
10.035142522452167


In [6]:
# Save each split to a separate file
train_data.to_parquet(os.path.join(data_save_folder_path, 'train_data.parquet'), index=False)
test_data.to_parquet(os.path.join(data_save_folder_path, 'test_data.parquet'), index=False)
val_data.to_parquet(os.path.join(data_save_folder_path, 'val_data.parquet'), index=False)

If you do not see the files in the folder structure then refresh the view.
 
**Now move on to training your model(s)!**